In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import time 
import torch
from torch import Tensor, empty
from typing import Optional, Tuple, Union, List
import helpers_functional as F
from helpers_layer import *

### Testing of Conv2d Layer

In [5]:
N = 1000
in_channels = 4
out_channels = 10
kernel_size = (2, 3)
padding = (2,3)
stride = 2

conv_pytorch = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
conv = Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
x = torch.randn((N, in_channels, 32, 32))

t1 = time.time()
dummy = conv_pytorch(x)
t2 = time.time()
expected = conv(x)
t3 = time.time()
actual = F.conv2d(x, conv.weight, conv.bias, padding=padding, stride=stride)
t4 = time.time()
actual2 = F.conv2d(x, conv_pytorch.weight, conv_pytorch.bias, padding=padding, stride=stride)

print("Expected.shape:", expected.shape)
print("Actual.shape:", actual.shape)

torch.testing.assert_allclose(actual, expected)
torch.testing.assert_allclose(actual2, dummy)

print("\nComputation Time Comparison:")
print("\tPytorch Conv2d: \t", t2-t1)
print("\tImplemented Conv2d: \t", t3-t2)
print("\tConv2d: \t\t", t4-t3)

Expected.shape: torch.Size([1000, 10, 18, 18])
Actual.shape: torch.Size([1000, 10, 18, 18])

Computation Time Comparison:
	Pytorch Conv2d: 	 0.0362701416015625
	Implemented Conv2d: 	 0.038072824478149414
	Conv2d: 		 0.036733150482177734


### Testing of ConvTranspose2d Layer

In [6]:
N = 100
in_channels = 4
out_channels = 10
kernel_size = (6, 9)
padding = (0,0)
stride = 1

convtranspose_pytorch = torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
convtranspose = ConvTranspose2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
x = torch.randn((N, in_channels, 32, 32))

t1 = time.time()
dummy = convtranspose_pytorch(x)
t2 = time.time()
expected = convtranspose(x)
t3 = time.time()
actual = F.conv_transpose2d(x, convtranspose.weight, convtranspose.bias, padding=padding, stride=stride)
t4 = time.time()
actual2 = F.conv_transpose2d(x, convtranspose_pytorch.weight, convtranspose_pytorch.bias, padding=padding, stride=stride)

print("Expected.shape:", expected.shape)
print("Actual.shape:", actual.shape)

torch.testing.assert_allclose(actual, expected)
torch.testing.assert_allclose(actual2, dummy)

print("\nComputation Time Comparison:")
print("\tPytorch Conv2d: \t", t2-t1)
print("\tImplemented Conv2d: \t", t3-t2)
print("\tConv2d: \t\t", t4-t3)

Expected.shape: torch.Size([100, 10, 37, 40])
Actual.shape: torch.Size([100, 10, 37, 40])

Computation Time Comparison:
	Pytorch Conv2d: 	 0.1106710433959961
	Implemented Conv2d: 	 0.18213391304016113
	Conv2d: 		 0.10601806640625
